In [ ]:
import csv
import random
import numpy as np

# ----- Death Probability Functions -----
def old_age_death_probability(age_months, A=0.001, B=0.0001, C=0.2, dt=1.0):
    """
    Compute the monthly death probability due solely to old age using an exponential hazard model.
    
    Parameters:
        age_months (float): Age of the rat in months.
        A, B, C (float): Model parameters.
        dt (float): Time step in months (default is 1 month).
        
    Returns:
        float: The probability of death due to old age in one timestep.
    """
    mu_x = A + B * np.exp(C * age_months)
    prob_death = 1 - np.exp(-mu_x * dt)
    return prob_death

def age_multiplier(age_months, old_age_death_func, dt=1.0):
    """
    Compute a multiplier to adjust death probabilities for non-age-related causes.
    
    For very young rats (<1 month), a multiplier of 3.0 is used because they are more vulnerable.
    For adolescents (1-18 months), the multiplier is baseline (1.0).
    For older rats, the multiplier increases slightly based on the ratio of the current age death
    probability to that at 12 months.
    
    Parameters:
        age_months (float): Age of the rat in months.
        old_age_death_func (function): Function to compute old age death probability.
        dt (float): Timestep in months.
        
    Returns:
        float: The age-based multiplier.
    """
    if age_months <= 1:
        return 3.0
    elif age_months <= 18:
        return 1.0
    else:
        base = old_age_death_func(12, dt=dt)
        current = old_age_death_func(age_months, dt=dt)
        return 1.0 + 0.1 * ((current / base) - 1)

def radiation_death_probability(
    age_months,
    planet_flux,  # Multiple of Earth's flux
    old_age_death_func,
    earth_annual_dose=0.0024,  # Sv/year
    lethal_dose=7.0,           # Sv, approximate LD50 for rats
    dt=1.0                     # timestep in months
):
    """
    Calculate the death probability due to radiation exposure over a month.
    
    The function computes the monthly radiation dose (in Sv) and calculates a baseline
    death probability from that dose. This probability is then multiplied by an age multiplier.
    
    Parameters:
        age_months (float): Age of the rat in months.
        planet_flux (float): Radiation flux relative to Earth.
        old_age_death_func (function): Function to compute old age death probability.
        earth_annual_dose (float): Earth's annual radiation dose (Sv/year).
        lethal_dose (float): Approximate LD50 for rats (in Sv).
        dt (float): Timestep in months.
        
    Returns:
        float: Final radiation death probability (capped at 1.0).
    """
    earth_monthly_dose = earth_annual_dose / 12  # Convert annual dose to monthly dose
    monthly_dose = earth_monthly_dose * planet_flux
    base_rad_prob = 1 - np.exp(-monthly_dose / lethal_dose)
    multiplier = age_multiplier(age_months, old_age_death_func, dt=dt)
    final_rad_prob = base_rad_prob * multiplier
    return min(final_rad_prob, 1.0)

def gravity_death_probability(
    age_months,
    planet_gravity,
    old_age_death_func,
    dt=1.0,
    safe_range=(0.5, 2.0),
    k=10.0  # Sensitivity parameter
):
    """
    Calculate the death probability due to non-optimal gravity conditions.
    
    If the planet's gravity is within the safe range, the base probability is 0.
    Otherwise, the probability increases with the deviation from the safe range.
    This base probability is then adjusted by an age multiplier.
    
    Parameters:
        age_months (float): Age of the rat in months.
        planet_gravity (float): Planet's gravity in Earth g units.
        old_age_death_func (function): Function to compute old age death probability.
        dt (float): Timestep in months.
        safe_range (tuple): Acceptable range of gravity (in g).
        k (float): Sensitivity parameter.
        
    Returns:
        float: Final gravity death probability (capped at 1.0).
    """
    if safe_range[0] <= planet_gravity <= safe_range[1]:
        base_gravity_prob = 0.0
    else:
        deviation = (safe_range[0] - planet_gravity) if planet_gravity < safe_range[0] else (planet_gravity - safe_range[1])
        base_gravity_prob = 1 - np.exp(-deviation / k)
    multiplier = age_multiplier(age_months, old_age_death_func, dt=dt)
    final_grav_prob = base_gravity_prob * multiplier
    return min(final_grav_prob, 1.0)

def temperature_death_probability(
    age_months,
    planet_temperature_C,  # Temperature in Celsius
    old_age_death_func,
    dt=1.0,
    a=1.5,         # Logistic steepness
    T_optimal=22,  # Optimal temperature for rats in °C
    T_threshold=15  # Temperature difference threshold before risk increases
):
    """
    Calculate the death probability due to non-optimal temperature.
    
    The function uses a logistic function based on the absolute difference between the
    ambient temperature (in °C) and the optimal temperature. The baseline probability is then
    scaled by an age multiplier.
    
    Parameters:
        age_months (float): Age of the rat in months.
        planet_temperature_C (float): Ambient temperature in Celsius.
        old_age_death_func (function): Function to compute old age death probability.
        dt (float): Timestep in months.
        a (float): Logistic function steepness.
        T_optimal (float): Optimal temperature for rats in °C.
        T_threshold (float): Temperature difference threshold.
        
    Returns:
        float: Final temperature death probability (capped at 1.0).
    """
    temp_diff = abs(planet_temperature_C - T_optimal)
    base_temp_prob = 1 / (1 + np.exp(-a * (temp_diff - T_threshold)))
    multiplier = age_multiplier(age_months, old_age_death_func, dt=dt)
    final_temp_prob = base_temp_prob * multiplier
    return min(final_temp_prob, 1.0)

def total_death_probability(age_months, planet_flux, planet_gravity, planet_temperature_C, old_age_death_func, dt=1.0):
    """
    Compute the overall monthly death probability by combining independent death probabilities.
    
    This function calculates the individual death probabilities due to old age, radiation,
    gravity, and temperature. Assuming these causes act independently, the overall survival
    probability is the product of their individual survival probabilities.
    
    Parameters:
        age_months (float): Age of the rat in months.
        planet_flux (float): Radiation flux relative to Earth.
        planet_gravity (float): Planet's gravity in g.
        planet_temperature_C (float): Ambient temperature in Celsius.
        old_age_death_func (function): Function to compute old age death probability.
        dt (float): Timestep in months.
        
    Returns:
        float: The overall death probability (capped at 1.0).
    """
    p_age = old_age_death_func(age_months, dt=dt)
    p_rad = radiation_death_probability(age_months, planet_flux, old_age_death_func, dt=dt)
    p_grav = gravity_death_probability(age_months, planet_gravity, old_age_death_func, dt=dt)
    p_temp = temperature_death_probability(age_months, planet_temperature_C, old_age_death_func, dt=dt)
    
    surv_age = 1 - p_age
    surv_rad = 1 - p_rad
    surv_grav = 1 - p_grav
    surv_temp = 1 - p_temp
    
    overall_survival = surv_age * surv_rad * surv_grav * surv_temp
    overall_death_prob = 1 - overall_survival
    print(overall_death_prob)  # Debug: print overall death probability
    return min(overall_death_prob, 1.0)

# ----- Exoplanet Class with Uncertainty Sampling -----
class Exoplanet:
    """
    Represents an exoplanet with various physical parameters.
    
    The class supports sampling uncertain parameters (mass, radius, orbit time, effective temperature)
    using provided uncertainties. It also computes the nominal surface gravity.
    """
    def __init__(self, name, mass, mass_unc, distance, distance_unit, radius, radius_unc,
                 orbit_time, orbit_time_unc, host_star, effective_temp,
                 effective_temp_unc_low, effective_temp_unc_high, radiation_flux, description):
        self.name = name
        self.mass = float(mass)
        self.mass_unc = float(mass_unc) if mass_unc not in ["NA", "", None] else 0.0
        self.distance = distance  # For reference only
        self.distance_unit = distance_unit  # For reference only
        self.radius = float(radius)
        self.radius_unc = float(radius_unc) if radius_unc not in ["NA", "", None] else 0.0
        self.orbit_time = float(orbit_time)
        self.orbit_time_unc = float(orbit_time_unc) if orbit_time_unc not in ["NA", "", None] else 0.0
        self.host_star = host_star  # For reference only
        self.effective_temp = float(effective_temp)
        self.effective_temp_unc_low = float(effective_temp_unc_low) if effective_temp_unc_low not in ["NA", "", None] else 0.0
        self.effective_temp_unc_high = float(effective_temp_unc_high) if effective_temp_unc_high not in ["NA", "", None] else 0.0
        self.radiation_flux = float(radiation_flux)
        self.description = description

    def sample_parameters(self):
        """
        Sample uncertain parameters using Gaussian distributions (for mass, radius, orbit time)
        and a triangular distribution for effective temperature if uncertainties are provided.
        
        Returns:
            dict: Sampled parameters.
        """
        sampled_mass = random.gauss(self.mass, self.mass_unc) if self.mass_unc > 0 else self.mass
        sampled_radius = random.gauss(self.radius, self.radius_unc) if self.radius_unc > 0 else self.radius
        sampled_orbit_time = random.gauss(self.orbit_time, self.orbit_time_unc) if self.orbit_time_unc > 0 else self.orbit_time
        if self.effective_temp_unc_low > 0 or self.effective_temp_unc_high > 0:
            lower_bound = self.effective_temp - self.effective_temp_unc_low
            upper_bound = self.effective_temp + self.effective_temp_unc_high
            sampled_effective_temp = random.triangular(lower_bound, upper_bound, self.effective_temp)
        else:
            sampled_effective_temp = self.effective_temp
        sampled_radiation_flux = self.radiation_flux

        return {
            "mass": sampled_mass,
            "radius": sampled_radius,
            "orbit_time": sampled_orbit_time,
            "effective_temp": sampled_effective_temp,
            "radiation_flux": sampled_radiation_flux
        }

    def gravity(self, mass=None, radius=None):
        """
        Calculate the nominal surface gravity in Earth g units.
        
        Parameters:
            mass (float): Optional mass to use (defaults to nominal mass).
            radius (float): Optional radius to use (defaults to nominal radius).
            
        Returns:
            float: Surface gravity (mass / radius^2).
        """
        if mass is None:
            mass = self.mass
        if radius is None:
            radius = self.radius
        return mass / (radius ** 2)

    def __str__(self):
        """
        Return a string representation of the exoplanet, including uncertainties and nominal gravity.
        """
        return (f"{self.name}: Mass={self.mass}±{self.mass_unc} Mₑ, Radius={self.radius}±{self.radius_unc} Rₑ, "
                f"Orbit={self.orbit_time}±{self.orbit_time_unc} days, Temp={self.effective_temp}K "
                f"(unc: -{self.effective_temp_unc_low}, +{self.effective_temp_unc_high}), "
                f"Radiation Flux={self.radiation_flux}, Gravity (nominal)={self.gravity():.2f} g\n"
                f"Description: {self.description}")

# ----- Rat Class with Biological Realism -----
class Rat:
    """
    Represents a rat with biological parameters and behavior.
    
    Each rat has energy reserves, a basal metabolic rate (BMR), and a set of traits (gravity and temperature tolerance).
    Rats can reproduce (with mutation) and are subject to death based on energy, old age, and environmental stressors.
    """
    def __init__(self, gravity_tolerance=1.0, temp_tolerance=1.0, age=0, weight=0.35, energy=1000):
        self.gravity_tolerance = gravity_tolerance
        self.temp_tolerance = temp_tolerance
        self.age = age                # Age in days
        self.weight = weight          # Weight in kg
        self.energy = energy          # Energy reserve in kcal
        self.alive = True

        # Randomly assign gender ("M" for male, "F" for female)
        self.gender = random.choice(["M", "F"])
        self.is_pregnant = False
        self.gestation_timer = 0

    def basal_metabolic_rate(self):
        """
        Compute the basal metabolic rate (BMR) using an allometric scaling law.
        
        Returns:
            float: BMR in kcal/day.
        """
        return 800 * (self.weight ** 0.75)

    def daily_energy_expenditure(self, planet_effective_temp, planet_gravity):
        """
        Compute the extra energy expenditure due to environmental stressors.
        
        This includes extra energy costs for deviations from Earth-like gravity and temperature,
        plus an additional cost if the rat is pregnant.
        
        Parameters:
            planet_effective_temp (float): Surface temperature in Kelvin.
            planet_gravity (float): Surface gravity in Earth g.
            
        Returns:
            float: Extra energy cost in kcal/day.
        """
        gravity_cost = abs(planet_gravity - self.gravity_tolerance) * 50
        ideal_temp = 288  # Ideal temperature (Earth average) in Kelvin
        temp_cost = abs(planet_effective_temp - ideal_temp) / 10
        pregnancy_cost = 20 if self.is_pregnant else 0
        return gravity_cost + temp_cost + pregnancy_cost

    def daily_activity(self, planet_effective_temp, planet_gravity, planet_flux):
        """
        Simulate one day of activity for the rat.
        
        This updates the rat's energy based on its energy gain (which depends on fitness)
        and energy expenditure (BMR plus environmental stress). It also updates the rat's age
        and checks for death using the combined death probability from old age, radiation,
        gravity, and temperature.
        
        Parameters:
            planet_effective_temp (float): Surface temperature in Kelvin.
            planet_gravity (float): Surface gravity in Earth g.
            planet_flux (float): Radiation flux relative to Earth.
        """
        if not self.alive:
            return

        # Calculate basal metabolic rate and extra energy expenditure.
        bmr = self.basal_metabolic_rate()
        extra_energy = self.daily_energy_expenditure(planet_effective_temp, planet_gravity)
        daily_energy_use = bmr + extra_energy

        # Compute fitness as a penalty based on deviation from ideal conditions.
        fitness_penalty = (abs(planet_gravity - self.gravity_tolerance) +
                           abs(planet_effective_temp - 288) / 100.0)
        fitness = max(0.1, 1 - fitness_penalty)
        daily_energy_gain = bmr * fitness

        # Compute net energy change and ensure it is not negative (unlimited food assumption).
        net_energy = daily_energy_gain - daily_energy_use
        if net_energy < 0:
            net_energy = 0

        self.energy += net_energy
        self.age += 1  # Age increases by one day

        # Handle gestation countdown for pregnant females.
        if self.is_pregnant:
            self.gestation_timer -= 1
            if self.gestation_timer <= 0:
                self.is_pregnant = False

        # Convert age in days to months (approx. 30 days per month)
        age_months = self.age / 30.0
        # Convert effective temperature (in Kelvin) to Celsius for death probability calculations.
        planet_temp_C = planet_effective_temp - 273.15

        # Calculate overall monthly death probability.
        death_prob = total_death_probability(age_months, planet_flux, planet_gravity, planet_temp_C, old_age_death_probability, dt=1)
        # If a random draw falls below the death probability, the rat dies.
        if random.random() < death_prob:
            self.alive = False

    def attempt_reproduction(self, potential_males):
        """
        Attempt reproduction for a female rat.
        
        A female rat that is not already pregnant and is at least 60 days old, with sufficient energy,
        may reproduce if at least one mature male is available. Reproduction consumes energy and produces
        a litter with slightly mutated tolerance traits.
        
        Parameters:
            potential_males (list): List of mature male rats.
            
        Returns:
            list: A list of offspring (empty if reproduction does not occur).
        """
        if self.gender != "F" or self.is_pregnant or self.age < 60:
            return []
        bmr = self.basal_metabolic_rate()
        if self.energy > 1.5 * bmr and potential_males:
            self.is_pregnant = True
            self.gestation_timer = 21  # Gestation period in days
            self.energy *= 0.8  # Energy cost for reproduction
            litter_size = random.randint(6, 12)
            offspring = []
            for _ in range(litter_size):
                mutation_factor = 0.05
                new_gravity_tolerance = self.gravity_tolerance * random.uniform(1 - mutation_factor, 1 + mutation_factor)
                new_temp_tolerance = self.temp_tolerance * random.uniform(1 - mutation_factor, 1 + mutation_factor)
                baby = Rat(new_gravity_tolerance, new_temp_tolerance, age=0, weight=self.weight, energy=self.energy * 0.5)
                offspring.append(baby)
            return offspring
        return []

# ----- Function to Convert Effective to Surface Temperature -----
def effective_to_surface(effective_temp, method="ratio"):
    """
    Convert effective temperature to surface temperature assuming an Earth-like greenhouse effect.
    
    Two methods are available:
    - "ratio": Multiply effective temperature by 288/255 (Earth's ratio of surface to effective temperature).
    - "delta": Add 33 K to the effective temperature.
    
    Parameters:
        effective_temp (float): Effective temperature in Kelvin.
        method (str): Conversion method ("ratio" or "delta").
        
    Returns:
        float: Estimated surface temperature in Kelvin.
    """
    if method == "ratio":
        greenhouse_factor = 288 / 255
        return effective_temp * greenhouse_factor
    elif method == "delta":
        return effective_temp + 33
    else:
        raise ValueError("Unknown method. Choose 'ratio' or 'delta'.")

# ----- Simulation Function for a Given Exoplanet -----
def run_simulation_on_planet(planet: Exoplanet, simulation_days=5*365, initial_population_size=5):
    """
    Run the evolutionary simulation for a given exoplanet.
    
    This function samples the exoplanet's uncertain parameters, converts the effective temperature
    to a surface temperature, computes the surface gravity, and then simulates the daily activities
    (energy usage, reproduction, death) of a rat population over a number of days.
    
    Parameters:
        planet (Exoplanet): The exoplanet object.
        simulation_days (int): Number of simulation days (default: 5 years).
        initial_population_size (int): Starting population size.
        
    Returns:
        list: The final list of surviving rats.
    """
    # Sample uncertain parameters from the exoplanet.
    sampled = planet.sample_parameters()
    sampled_mass = sampled["mass"]
    sampled_radius = sampled["radius"]
    sampled_orbit_time = sampled["orbit_time"]
    sampled_effective_temp = sampled["effective_temp"]
    sampled_radiation_flux = sampled["radiation_flux"]
    
    # Convert effective temperature to surface temperature.
    sampled_surface_temp = effective_to_surface(sampled_effective_temp, method="ratio")
    # Calculate surface gravity using the sampled mass and radius.
    sampled_gravity = sampled_mass / (sampled_radius ** 2)
    
    # Print the sampled parameters.
    print(f"Running simulation for {planet.name} with sampled parameters:")
    print(f"  Mass = {sampled_mass:.3f} Mₑ, Radius = {sampled_radius:.3f} Rₑ, Orbit Time = {sampled_orbit_time:.3f} days")
    print(f"  Effective Temp = {sampled_effective_temp:.1f} K, Surface Temp = {sampled_surface_temp:.1f} K, Radiation Flux = {sampled_radiation_flux}, Gravity = {sampled_gravity:.2f} g\n")
    
    # Initialize the rat population.
    population = [Rat() for _ in range(initial_population_size)]
    
    # Run the simulation day-by-day.
    for day in range(simulation_days):
        new_offspring = []
        # Filter out mature male rats (at least 60 days old) for reproduction.
        alive_males = [rat for rat in population if rat.alive and rat.gender == "M" and rat.age >= 60]
        for rat in population:
            if rat.alive:
                # Each rat performs its daily activities.
                rat.daily_activity(sampled_surface_temp, sampled_gravity, sampled_radiation_flux)
                # Females attempt reproduction if conditions are met.
                if rat.gender == "F":
                    offspring = rat.attempt_reproduction(alive_males)
                    if offspring:
                        new_offspring.extend(offspring)
        # Add new offspring to the population.
        population.extend(new_offspring)
        # Remove dead rats from the population.
        population = [rat for rat in population if rat.alive]
        # Print population status every day.
        if day % 1 == 0:
            avg_energy = sum(r.energy for r in population) / len(population) if population else 0
            print(f"Day {day} on {planet.name}: Population = {len(population)}, Average Energy = {avg_energy:.1f} kcal")
    
    # Print final average tolerance values if any rats survive.
    if population:
        avg_gravity_tolerance = sum(r.gravity_tolerance for r in population) / len(population)
        avg_temp_tolerance = sum(r.temp_tolerance for r in population) / len(population)
        print(f"\nAfter {simulation_days} days on {planet.name}:")
        print(f"  Average Gravity Tolerance: {avg_gravity_tolerance:.2f}")
        print(f"  Average Temperature Tolerance: {avg_temp_tolerance:.2f}")
    else:
        print(f"No survivors on {planet.name} after simulation.")
    
    return population

# ----- Read CSV and Run Simulation for Each Exoplanet -----
exoplanets = []
with open('exoplanets.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            # Create an Exoplanet object for each row in the CSV.
            planet = Exoplanet(
                name=row["Name"],
                mass=row["Mass"],
                mass_unc=row["Mass_unc"],
                distance=row["Distance"],
                distance_unit=row["Distance_unit"],
                radius=row["Radius"],
                radius_unc=row["Radius_unc"],
                orbit_time=row["Orbit_Time"],
                orbit_time_unc=row["Orbit_Time_unc"],
                host_star=row["Host_Star"],
                effective_temp=row["Effective_Temp"],
                effective_temp_unc_low=row["Effective_Temp_unc_low"],
                effective_temp_unc_high=row["Effective_Temp_unc_high"],
                radiation_flux=row["Radiation_Flux"],
                description=row["Description"]
            )
            exoplanets.append(planet)
        except Exception as e:
            print(f"Error processing row: {row}\n{e}")

# Run the simulation for each exoplanet.
for planet in exoplanets:
    print("\n-------------------------------------------")
    print(planet)
    survivors = run_simulation_on_planet(planet)



-------------------------------------------
Proxima Centauri B: Mass=1.07±0.06 Mₑ, Radius=0.92±0.012 Rₑ, Orbit=11.1868±0.003 days, Temp=234.0K (unc: -14.0, +6.0), Radiation Flux=1.0, Gravity (nominal)=1.26 g
Description: Positioned comfortably within the habitable zone; if it retains an atmosphere, its temperate conditions make it a strong candidate for habitability studies.
Running simulation for Proxima Centauri B with sampled parameters:
  Mass = 1.041 Mₑ, Radius = 0.903 Rₑ, Orbit Time = 11.182 days
  Effective Temp = 239.0 K, Surface Temp = 270.0 K, Radiation Flux = 1.0, Gravity = 1.28 g

1.0
1.0
1.0
1.0
1.0
Day 0 on Proxima Centauri B: Population = 0, Average Energy = 0.0 kcal
Day 1 on Proxima Centauri B: Population = 0, Average Energy = 0.0 kcal
Day 2 on Proxima Centauri B: Population = 0, Average Energy = 0.0 kcal
Day 3 on Proxima Centauri B: Population = 0, Average Energy = 0.0 kcal
Day 4 on Proxima Centauri B: Population = 0, Average Energy = 0.0 kcal
Day 5 on Proxima Centaur